# Homework 1
By Quentin Barnes  
CS 344

### Part one: Introspection:
Introspection is the practice of looking inward, towards one’s own thoughts in an attempt to gain insight on how we work.  It is a practice that is as old as Plato and is very useful in the field of psychology.  Because of this, it is useful in modeling for AI, however only to an extent.  It is useful because a lot of the times we can teach the computers to solve the problems the same ways that we do.  This does have some limitations and drawbacks, though, since computers are mechanical in nature.  Sometimes the way that we do things in our heads is very inefficient because we do have limitations.  
  
I think that an interesting area to look at is vision.  As we are teaching computers to actually see, we are doing it in a way that is similar to how we learned to see.  For computers we have people label a bunch of objects and then feed it to programs to let them learn what those objects look like, which is a lot like how humans learn to recognize things.  However, even in this case, they don’t do it exactly the same, and the program can also be fooled a lot easier.  

### Part 2: Traveling Salesman Probelm: 

In [8]:
"""
This module implements local search on the Traveling Salesman Problem.  It uses
simulated annealing and hill climbing to do this.  It will randomly create a city
that has a list of linked cities in it.  It will then use the above algorithms to
try and find the shortest path through them.

@author: Quentin Barnes
"""
import math
import time
from search import Problem, hill_climbing, simulated_annealing, \
    exp_schedule, genetic_search
from random import randrange, shuffle


class TPSVariant(Problem):
    """
    State: Current order of citys
    Action: Take the city with the longest distance to the next one, then make a list
    of all the other cities swaped with that one.
    Value: Total distance took to travel between the cities
    """

    # Sets the initial state, the size of the city, the configuration of the cities, and the maximum distance
    def __init__(self, initial, citySize, cityConfig, maximum, connectHome):
        self.initial = initial[:]
        self.citySize = citySize
        self.cityConfig = cityConfig
        self.maximum = maximum
        self.connectHome = connectHome

    # It takes the city with the max distance to the next one and then
    # makes a list of that city swaped with each other city as the actions
    def actions(self, state):
        max = self.getMaxDistNode(state)
        options = []
        for i in range(self.citySize):
            if i != max:
                newOption = state[:]
                newOption[i], newOption[max] = newOption[max], newOption[i]
                options.append(newOption[:])
        return options[:]

    # Result is the action taken
    def result(self, state, action):
        return action[:]

    # Inverts the realValue since the algorithms are looking for a maximum,
    # while we are looking for the minimum distance
    def value(self, state):
        return self.maximum - self.realValue(state)

    # Uses the city configuration to sum up the total distance that the salesman travels
    def realValue(self, state):
        totalDist = 0
        for i in range(self.citySize - 1):
            totalDist += self.cityConfig[state[i]][state[i + 1]]
        if self.connectHome:
            totalDist += self.cityConfig[self.citySize-1][1]
        return totalDist

    # Finds and returns the index of the node with the max distance
    def getMaxDistNode(self, state):
        indexOfMax = 0
        lengthOfMax = 0
        for n in range(self.citySize - 1):
            if self.cityConfig[state[n]][state[n + 1]] > lengthOfMax:
                indexOfMax = n
                lengthOfMax = self.cityConfig[state[n]][state[n + 1]]
        return indexOfMax


if __name__ == '__main__':

    # Sets the values of how large the city is and the max distance between them
    # Also sets if the problem should be a loop or not
    fillInSize = citySize = 15
    maxDistance = 20
    connectHome = False

    # Creates the city configuration
    maximum = citySize * maxDistance
    cityConfig = [[0 for x in range(citySize)] for y in range(citySize)]
    for i in range(citySize):
        fillInSize -= 1
        for j in range(fillInSize):
            cityConfig[i + j + 1][i] = cityConfig[i][i +
                                                     j + 1] = randrange(1, maxDistance, 1)

    # Creates an initial state
    initial = [x for x in range(citySize)]
    shuffle(initial)

    p = TPSVariant(initial, citySize, cityConfig, maximum, connectHome)

    time1 = time.time()
    hill_solution = hill_climbing(p)
    time2 = time.time()

    simAneal = simulated_annealing(
        p,
        exp_schedule(k=20, lam=0.005, limit=1000)
    )
    time3 = time.time()

    print("Initial: \t\t", initial, "  Distance: ", p.realValue(initial))
    print("Hill Climbing solution: ", hill_solution,
          "  Distance: ", p.realValue(hill_solution), "  Time: ", (time2 - time1))
    print("Sim Anneal solution: \t", simAneal,
          "  Distance: ", p.realValue(simAneal), "  Time: ", (time3 - time2))


Initial: 		 [7, 3, 14, 13, 6, 0, 2, 9, 11, 4, 10, 1, 8, 12, 5]   Distance:  114
Hill Climbing solution:  [13, 3, 9, 14, 6, 0, 2, 7, 11, 4, 10, 1, 8, 12, 5]   Distance:  77   Time:  0.0005633831024169922
Sim Anneal solution: 	 [10, 6, 0, 7, 3, 14, 1, 12, 8, 11, 13, 4, 2, 9, 5]   Distance:  71   Time:  0.06310081481933594


 This program is creating a random map and then uses hill climbing and simulated annealing to try and solve the problem.  The states for the problem are lists of the order that the cities will be traveled to.  It was unclear whether the salesman was supposed to travel back to where he started in the end or not, so I made it an option for the program.  The actions that I chose were for the state given, find the longest path to the next city, and then make the actions variations of the state with the that city swapped for each other city.  This seemed to give me pretty good results, even with the hill climbing algorithm.  I also tried to just shuffle the state and give that back as an action, and while it didn’t work too well for hill climbing, it did work surprisingly well for simulated annealing.  This is most likely because it had a lot of random states to choose the answer from.  In terms of which algorithm worked the best, simulated annealing was more consistent, and on average it produced better results.  I implemented random restarts just to check this, and simulated annealing did show to be consistently faster.  This is most likely because this problem has a lot of good solutions, so it’s easy for both algorithms to find decent solutions.  However, hill climbing still can get stuck, so simulated annealing turned out to be better.

### Part 3: CSP:

In [9]:
"""
This program uses CSP to schedual classes for a cs department with
2 classrooms, 4 teachers, 5 timeslots, and 7 classes.

@author Quentin
"""
from csp import CSP, min_conflicts, parse_neighbors
import time

from csp import min_conflicts, backtracking_search, AC3, CSP, parse_neighbors
from search import depth_first_graph_search


def courseScheduling():
    # Set up domains and variables
    Rooms = 'GoldLab MaroonLab'.split()
    Teachers = 'Schuurman Adams Norman VanderLinden'.split()
    Times = 'MWF800 MWF900 MWF1030 TTH1030 TTH1200'.split()
    variables = 'CS108 CS112 CS212 CS214 CS344 CS300 CS384'.split()

    options = [0 for i in range(40)]
    i = 0

    for teacher in Teachers:
        for time in Times:
            for room in Rooms:
                options[i] = [teacher, time, room]
                i += 1

    domains = {}
    for var in variables:
        domains[var] = options

    # Create the neighbors of the other classes
    neighbors = parse_neighbors("""
        CS108: CS112 CS212 CS214 CS344 CS300 CS384;
        CS112: CS212 CS214 CS344 CS300 CS384;
        CS212: CS214 CS344 CS300 CS384;
        CS214: CS344 CS300 CS384;
        CS344: CS300 CS384;
        CS300: CS384""")

    def constraints(A, a, B, b):

        # Are they the same course
        if A == B:
            return False
        # Who teaches each course
        elif A == 'CS112' and a[0] != 'Adams':
            return False
        elif B == 'CS112' and b[0] != 'Adams':
            return False
        elif A == 'CS212' and a[0] != 'Adams':
            return False
        elif B == 'CS212' and b[0] != 'Adams':
            return False
        elif A == 'CS214' and a[0] != 'Norman':
            return False
        elif B == 'CS214' and b[0] != 'Norman':
            return False
        elif A == 'CS344' and a[0] != 'VanderLinden':
            return False
        elif B == 'CS344' and b[0] != 'VanderLinden':
            return False
        elif A == 'CS108' and a[0] != 'VanderLinden':
            return False
        elif B == 'CS108' and b[0] != 'VanderLinden':
            return False
        elif A == 'CS384' and a[0] != 'Schuurman':
            return False
        elif B == 'CS384' and b[0] != 'Schuurman':
            return False
        elif A == 'CS300' and a[0] != 'Schuurman':
            return False
        elif B == 'CS300' and b[0] != 'Schuurman':
            return False
        # is the professor already teaching at a time
        elif a[0] == b[0] and a[1] == b[1]:
            return False
        # is the classroom already being used
        elif a[1] == b[1] and a[2] == b[2]:
            return False
        else:
            return True

    return CSP(variables, domains, neighbors, constraints)


puzzle = courseScheduling()

# result = depth_first_graph_search(puzzle)
# result = AC3(puzzle)
# result = backtracking_search(puzzle)
result = min_conflicts(puzzle, max_steps=1000)

if result is not None:
    print("Solution:\n", result)
else:
    print("failed...")
    print(puzzle.curr_domains)
    puzzle.display(puzzle.infer_assignment())


Solution:
 {'CS108': ['VanderLinden', 'MWF800', 'GoldLab'], 'CS112': ['Adams', 'TTH1030', 'GoldLab'], 'CS212': ['Adams', 'MWF1030', 'GoldLab'], 'CS214': ['Norman', 'MWF900', 'GoldLab'], 'CS344': ['VanderLinden', 'TTH1030', 'MaroonLab'], 'CS300': ['Schuurman', 'TTH1200', 'MaroonLab'], 'CS384': ['Schuurman', 'MWF800', 'MaroonLab']}


I chose to make my variables the courses because I thought it would be easier to make sure they only happened once.  Then I made a list of all the possibilities that the course could have and used the constraints to narrow down the options.  I found that writing out my constraints was easy but converting my constraints to logic was very hard.  